# Database connectivity 

in Python refers to the ability of Python applications to communicate and interact with databases. This is usually achieved using modules or libraries that provide a consistent interface between the Python application and the database system.

In [ ]:
import sqlite3

db_path = 'C:/work_prog/GEA_IKV_Prog_2/notebooks/sample.db'

# Step 1: Connect to a database named 'sample.db'. If it doesn't exist, it will be created.
conn = sqlite3.connect(db_path)

# Step 2: Create a table called `users` with an `id` and a `name`.
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
)
''')

# Step 3: Populate the table with some data.
users_data = [
    ('Alenka',),
    ('Boris',),
    ('Cene',),
]
cursor.executemany('INSERT INTO users (name) VALUES (?)', users_data)
conn.commit()

# Step 4: Query the table and retrieve all rows.
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()

# Step 5: Close the connection.
conn.close()

rows


Demo of CRUD (Create, Read, Update, Delete) operations

In [ ]:
import sqlite3

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def create_user(conn, name):
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (name) VALUES (?)", (name,))
    conn.commit()
    return cursor.lastrowid

def read_users(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    return cursor.fetchall()

def read_single_user(conn, user_id):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE id = " + user_id) # SELECT * FROM Users WHERE UserId = 105 OR 1=1;
    return cursor.fetchall()

def update_user(conn, user_id, new_name):
    cursor = conn.cursor()
    cursor.execute("UPDATE users SET name = ? WHERE id = ?", (new_name, user_id))
    conn.commit()

def delete_user(conn, user_id):
    cursor = conn.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    conn.commit()


In [ ]:
# Connect to the database
conn = connect_to_db(db_path)
    
# Create a new user
new_user_id = create_user(conn, "Domen")
print(f"User 'Domen' added with ID {new_user_id}")
    
# Read all users
print("Users in the database:")
for user in read_users(conn):
    print(user)
    
# Update a user's name
update_user(conn, new_user_id, "Damjan")
print(f"\nUpdated user {new_user_id}'s name to 'Damjan'")
    
# Read all users after update
print("Users in the database after update:")
for user in read_users(conn):
    print(user)
    
# Delete a user
delete_user(conn, new_user_id)
print(f"\nDeleted user with ID {new_user_id}")
    
# Read all users after deletion
print("Users in the database after deletion:")
for user in read_users(conn):
    print(user)
    
# Close the connection
conn.close()


# SQL injection attack example

In [ ]:
# Connect to the database
conn = connect_to_db(db_path)
    
# Read user with id 1
user_id = "1"
print("User with id 1 is:")
for user in read_single_user(conn, user_id):
    print(user)

print()

# Read user with id 1 with "injection"
injection = " OR 1=1"
user_id = user_id + injection
print("User(s) with id 1 -- with SQL injection -- are:")
for user in read_single_user(conn, user_id):
    print(user)
    
# Close the connection
conn.close()


In [ ]:
import os

# delete db file
os.remove(db_path)

# Connect to postgres database

In [ ]:
import psycopg2

def connect_to_postgres(dbname, user, password, host="localhost", port="5432"):
    try:
        # Connect to the PostgreSQL server
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        
        # Create a new cursor
        cursor = conn.cursor()
        
        # Print PostgreSQL version
        cursor.execute("SELECT version();")
        db_version = cursor.fetchone()
        print(f"Connected to: {db_version[0]}")
        
        # Close the cursor and connection
        cursor.close()
        conn.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        return None

# Call the function
connect_to_postgres("gea_ikv_test", "bojan", "bojan123", "localhost", "5432")


In [ ]:
import pandas as pd

def connect_and_fetch_data():
    # PostgreSQL server connection parameters
    host = "localhost"  
    dbname = "ashrae"
    user = "bojan"
    password = "bojan123"
    port = "5432"
    
    # Connection string
    conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"

    try:
        # Connect to the PostgreSQL server
        conn = psycopg2.connect(conn_string)

        # Create a new cursor
        cur = conn.cursor()

        # Execute a query
        cur.execute("SELECT * FROM breast_cancer")

        # Fetch all rows from the table
        rows = cur.fetchall()

        # Convert to a pandas DataFrame for nicer display
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])

        # Display the DataFrame
        print(df)

        # Close the cursor and connection
        cur.close()
        conn.close()
        return df

    except Exception as e:
        print(f"An error occurred: {e}")

df = connect_and_fetch_data()


In [ ]:
from tabulate import tabulate

# Use tabulate to display the DataFrame in a table format
print(tabulate(df, headers='keys', tablefmt='grid'))


In [ ]:
# Example of using pandas styling
styled_df = df.style.background_gradient(cmap='viridis', subset=["deg_malig"]) \
    .highlight_max(color='lightgreen', subset=["tumor_size"]) \
    .highlight_min(color='yellow', subset=["class_name"]) \
    .format("{:.2f}", subset=[]) \
    .set_properties(**{'text-align': 'center'})

# Display the styled DataFrame
styled_df
